In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import os

from torchvision.models.resnet import (
    ResNet18_Weights,
)

path = "./data"

transform = transforms.Compose([transforms.Resize((200, 200)), transforms.ToTensor()])
dataset_combined = datasets.ImageFolder(root=path, transform=transform)


# Splitting the dataset into train, validation, and test sets
total_size = len(dataset_combined)
test_size = int(0.2 * total_size)
train_size = total_size - test_size
train_dataset, test_dataset = random_split(dataset_combined, [train_size, test_size])

# Further split the training set into train and validation sets
val_size = int(0.15 * train_size)
train_size = train_size - val_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


# Define the ResNet model
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        # Load a pre-trained resnet model
        self.resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        # Replace the classifier
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(
            num_features, 2
        )  # Binary classification (fire, non-fire)

    def forward(self, x):
        return self.resnet(x)


# Initialize the model
model = ResNetModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Adding the training loop and validation steps to the ResNet model training code

# Assuming the rest of the setup as in the previous code snippet

# Number of epochs
num_epochs = 10


# Function to train and validate the model
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    train_acc_history = []
    val_acc_history = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        # Training phase
        image_counter = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # Update the image counter
            image_counter += inputs.size(0)

            # Check if the counter has reached a multiple of 100
            if image_counter % 100 == 0:
                print(
                    f"Processed {image_counter} images, Loss: {running_loss/image_counter}, Accuracy: {running_corrects/image_counter}"
                )

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(
            f"Epoch {epoch}/{num_epochs - 1} - Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}"
        )

        train_acc_history.append(epoch_acc.item())
        val_acc_history.append(val_acc.item())

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

            val_loss = val_loss / len(val_loader.dataset)
            val_acc = val_corrects.double() / len(val_loader.dataset)

        print(
            f"Epoch {epoch}/{num_epochs - 1} - Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}"
        )
    return train_acc_history, val_acc_history


import matplotlib.pyplot as plt

# Call the train_model function and get the history
train_acc_history, val_acc_history = train_model(
    model, criterion, optimizer, train_loader, val_loader, num_epochs
)

# Plotting the accuracies
plt.figure(figsize=(10, 5))
plt.plot(train_acc_history, label="Train Accuracy")
plt.plot(val_acc_history, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy over Epochs")
plt.legend()
plt.show()


Processed 800 images, Loss: 0.5771452152729034, Accuracy: 0.7599999904632568
Processed 1600 images, Loss: 0.4799711801111698, Accuracy: 0.7987499833106995
Processed 2400 images, Loss: 0.4305359678467115, Accuracy: 0.8245833516120911
Processed 3200 images, Loss: 0.4067348635941744, Accuracy: 0.8353124856948853
Processed 4000 images, Loss: 0.3810852805972099, Accuracy: 0.8450000286102295
Processed 4800 images, Loss: 0.3869293242196242, Accuracy: 0.8420833349227905
Processed 5600 images, Loss: 0.3730095673033169, Accuracy: 0.8473214507102966
Processed 6400 images, Loss: 0.36968141082674266, Accuracy: 0.8495312333106995
Processed 7200 images, Loss: 0.36428737653626336, Accuracy: 0.8530555367469788
Processed 8000 images, Loss: 0.359433967590332, Accuracy: 0.8551250100135803
Processed 8800 images, Loss: 0.3481286093321714, Accuracy: 0.8597727417945862
Processed 9600 images, Loss: 0.34043016004065674, Accuracy: 0.8630208373069763
Processed 10400 images, Loss: 0.3321604415774345, Accuracy: 0.8

KeyboardInterrupt: 